In [24]:
import pandas as pd
dataset = input("Enter the path to the dataset CSV file: ")
concrete = pd.read_csv(dataset)
dependent = input("Enter the name of the dependent column: ")
concrete_temp = concrete.copy()
X = concrete_temp.drop(dependent, axis=1).copy()
# Now, try accessing the column again
y = concrete[dependent]
# Replace 'your_file.csv' with the actual path to your CSV file
frequent_itemset_file_path = input("Enter the path to the frequent itemset CSV file: ")
# Define your column names
column_names = ['Frequent_Itemsets_CHARM']
# Read the CSV file into a DataFrame with specified column names
freq_item = pd.read_csv(frequent_itemset_file_path, names=column_names)
one_hot_encoded = pd.get_dummies(freq_item['Frequent_Itemsets_CHARM'].str.split(expand=True), prefix='', prefix_sep='')
from sklearn.impute import SimpleImputer
def handle_missing_values(dataset):
    # Check if there are missing values in the dataset
    if dataset.isnull().any().any():
        # Identify numerical and categorical columns
        numerical_cols = dataset.select_dtypes(include=['number']).columns
        categorical_cols = dataset.select_dtypes(exclude=['number']).columns

        # Impute numerical columns with mean
        imputer_numeric = SimpleImputer(strategy='mean')
        dataset[numerical_cols] = imputer_numeric.fit_transform(dataset[numerical_cols])

        # Impute categorical columns with the most frequent value (mode)
        imputer_categorical = SimpleImputer(strategy='most_frequent')
        dataset[categorical_cols] = imputer_categorical.fit_transform(dataset[categorical_cols])

        return dataset
    else:
        # If no missing values, return the original dataset
        return dataset
X = handle_missing_values(X)
def binarize_dataframe(input_df, num_bins):
    df = pd.DataFrame(input_df)
    df_binarized = pd.DataFrame()

    for column in df.columns:
        column_name = column+'_binarized'
        bins = pd.qcut(df[column], q=num_bins, labels=False, duplicates='drop')
        df_binarized[column_name] = (bins == bins.max()).astype(int)

    return df_binarized
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

def create_intervals_df(data_frame, column_name, num_intervals):
    # Extract the specified column
    value_ag = data_frame[column_name]

    #print("Extracted column values:")
    #print(value_ag)

    # Check if the column contains numeric data
    if not pd.api.types.is_numeric_dtype(value_ag):
        raise ValueError(f"The '{column_name}' column must contain numeric data.")

    # Check for missing values in the column
    if value_ag.isnull().any():
        raise ValueError(f"The '{column_name}' column contains missing values. Please handle them before processing.")

    # Check if scaling and clipping is necessary
    if value_ag.min() < 0 or value_ag.max() > 1:
        # Rescale to the range [0, 1]
        value_ag_scaled = (value_ag - value_ag.min()) / (value_ag.max() - value_ag.min())

        # Clip values to the range [0, 1]
        value_ag_scaled_clipped = np.clip(value_ag_scaled, 0, 1)
    else:
        # No need to rescale or clip
        value_ag_scaled_clipped = value_ag

    #print("Scaled and clipped column values:")
    #print(value_ag_scaled_clipped)

    # Calculate bin edges dynamically within the [0, 1] range
    bin_edges = np.linspace(0, 1, num=num_intervals + 1)

    # Create intervals and assign labels for clipped values
    data_inter_clipped = pd.cut(value_ag_scaled_clipped.values, bins=bin_edges, labels=[chr(ord('A') + i) for i in range(len(bin_edges) - 1)])

    #print("Computed intervals:")
    #print(data_inter_clipped)

    # Store intervals in a DataFrame
    intervals_df = pd.DataFrame({
        'Category': data_inter_clipped.categories,
        'Min_Value': [value_ag_scaled_clipped[data_inter_clipped == category].min() for category in data_inter_clipped.categories],
        'Max_Value': [value_ag_scaled_clipped[data_inter_clipped == category].max() for category in data_inter_clipped.categories]
    })

    return intervals_df
num_intervals = 10
dependent_column_name = dependent
intervals_df = create_intervals_df(concrete, dependent_column_name, num_intervals)
def categorize_column(dataset, dependent_column_name, num_intervals):
    # Fill NaN values with the mean of the column
    dataset_filled = dataset.fillna(dataset[dependent_column_name].mean())

    # Extract the specified dependent column
    value_ag_filled = dataset_filled[dependent_column_name]

    # Define the target range (0-1)
    target_range = (0, 1)

    # Scale to the target range [0, 1] using np.interp
    value_ag_scaled_filled = np.interp(value_ag_filled, (value_ag_filled.min(), value_ag_filled.max()), target_range)

    # Create intervals and assign labels for scaled values
    data_inter_filled = pd.cut(value_ag_scaled_filled, bins=num_intervals, labels=[chr(ord('A') + i) for i in range(num_intervals)])

    # Store intervals in a DataFrame
    result_df = pd.DataFrame({
        'Category': data_inter_filled,
        dependent_column_name: value_ag_scaled_filled
    })

    return result_df
num_intervals = 10
result_df = categorize_column(concrete, dependent, num_intervals)
#Data Binary
df = pd.DataFrame(concrete)
num_bins = 15
# Use the previously defined binarize_dataframe function
df_binarized = binarize_dataframe(df, num_bins)
dependent_column_name = dependent
# Rename columns and create 'Data_binary'
df_binarized.columns = df_binarized.columns.str.replace('_binarized', '')
data_incidence_input = df_binarized.drop(columns=[dependent_column_name])
Data_binary = data_incidence_input.copy()
Data_binary['Category'] = result_df['Category']
Data_binary[dependent_column_name] = df_binarized[dependent_column_name]
#Jaccard similarity
import pandas as pd
import numpy as np

def calculate_jaccard_similarity(Data_binary, one_hot_encoded):
    # Number of rows in the Data_binary DataFrame
    num_rows_binary = Data_binary.shape[0]

    # Number of rows in the one_hot_encoded DataFrame
    num_rows_encoded = one_hot_encoded.shape[0]

    # Number of columns in the one_hot_encoded DataFrame
    num_cols_encoded = one_hot_encoded.shape[1]

    # Creating a matrix to store Jaccard similarities
    result_matrix = np.zeros((num_rows_binary, num_rows_encoded))

    # Calculating Jaccard similarity for each cell (pair of row in Data_binary and row in one_hot_encoded)
    for i in range(num_rows_binary):
        for j in range(num_rows_encoded):
            set_row_binary = set(Data_binary.iloc[i, 1:])  # Exclude the 'Category' column
            set_row_encoded = set(one_hot_encoded.iloc[j, :])
            intersection_size = len(set_row_binary.intersection(set_row_encoded))
            union_size = len(set_row_binary.union(set_row_encoded))
            similarity = intersection_size / union_size if union_size != 0 else 0
            result_matrix[i, j] = similarity

    # Convert the result matrix to a DataFrame
    result_df = pd.DataFrame(result_matrix, columns=one_hot_encoded.index)

    return result_df
jaccard_result = calculate_jaccard_similarity(Data_binary, one_hot_encoded)
#Transform Matrix
def transform_matrix(matrix):
    # Create a copy of the input matrix
    transformed_matrix = matrix.copy()

    # Apply the transformation
    transformed_matrix[transformed_matrix > 0.50] = 1
    transformed_matrix[transformed_matrix <= 0.50] = 0
    return transformed_matrix
transformed_matrix = transform_matrix(jaccard_result)
from scipy.sparse import csr_matrix
SXmat = csr_matrix(transformed_matrix)
from scipy.sparse import dok_matrix
def train_GIS_single_label(SX):
    """
    Trains the GIS algorithm for parameter estimation based on the sparse incidence matrix SX.
    """
    # Calculate the observed supports
    OS = np.array(SX.sum(axis=0)).flatten()

    # Initialize the weights and expected supports
    w = np.zeros(SX.shape[1], dtype=np.float32)
    ES = dok_matrix((1, SX.shape[1]), dtype=np.float32)

    # Iterate until convergence
    loop = True
    while loop:
        # Calculate the probabilities
        P = np.exp(SX.dot(w))

        # Update the expected supports
        ES = dok_matrix((1, SX.shape[1]), dtype=np.float32)
        for i in range(SX.shape[0]):
            P[i] /= np.sum(P)  # Normalize the probabilities
            ES += P[i] * SX[i]

        # Update the weights
        wold = w.copy()
        w += np.log(OS) - np.log(ES.toarray().flatten())  # Convert ES to a 1D array for element-wise operations
        wnorm = np.sum(np.exp(w))
        w /= wnorm  # Normalize the weights
        wprecision = np.max(np.abs(w - wold))
        if wprecision < 0.01:
            loop = False

    return w, wnorm
#Weights and wnorm for category
def calculate_weights_and_wnorm_for_category(category, work1_filled, one_hot_encoded):
    # Calculate Jaccard similarity
    jaccard_similarity_matrix = calculate_jaccard_similarity(Data_binary, one_hot_encoded)

    # Transform the Jaccard similarity matrix
    transformed_matrix = transform_matrix(jaccard_similarity_matrix)

    # Convert data to a scipy sparse format
    SXmat = csr_matrix(transformed_matrix)

    # Train GIS for the category using the transformed matrix
    w, wnorm = train_GIS_single_label(SXmat)

    return w, wnorm

# Iterate over categories in the 'Category' column of Work1_filled
categories_weights = {}

# Assuming 'Category' is the column containing category labels in 'Work1_filled'
categories = Data_binary['Category'].unique()

for category in categories:
    # Calculate weights and wnorm for the current category
    weights, wnorm = calculate_weights_and_wnorm_for_category(category, Data_binary, one_hot_encoded)

    # Store the weights and wnorm for the category
    categories_weights[category] = {'weights': weights, 'wnorm': wnorm}
def calculate_x_by_c(X, one_hot_encoded, category_weights):
    """
    Calculate x_by_c for each row in the dataset.

    Parameters:
    - X: DataFrame containing the dataset.
    - one_hot_encoded: DataFrame containing the one-hot-encoded matrix.
    - category_weights: Dictionary containing weights and wnorm for each category.

    Returns:
    - x_by_c: DataFrame with x_by_c values for each row.
    """

    # Calculate Jaccard similarity matrix for the entire dataset once
    jaccard_result = calculate_jaccard_similarity(X, one_hot_encoded)

    # Create an empty DataFrame to store x_by_c values
    x_by_c = pd.DataFrame(index=X.index)

    # Iterate over each row in the dataset
    for index, row in X.iterrows():
        try:
            # Extract the category for the current row
            category = row['Category']

            # Extract only the row corresponding to the current row in X
            jaccard_row = jaccard_result.loc[index]

            # Transform the row to a 1D array
            jaccard_row_array = jaccard_row.values.reshape(1, -1)

            # Get weights and wnorm for the current category from the provided dictionary
            category_info = category_weights.get(category)
            if category_info is None:
                raise ValueError(f"No weights and wnorm found for category: {category}")
            weights = category_info['weights']
            wnorm = category_info['wnorm']
            weights = np.expand_dims(weights, axis=0)  # Ensure weights is a row vector

            # Perform matrix multiplication, ensuring shapes are aligned
            hm = np.dot(jaccard_row_array, weights.T)

            row_value = np.squeeze(hm) + wnorm

            # Set the entire column at once using .loc
            x_by_c.loc[index, 'P(x/c)'] = row_value
        except ValueError as e:
            print(f"Error in row {index}: {e}")
            raise

    return x_by_c
x_by_c_values = calculate_x_by_c(Data_binary, one_hot_encoded, categories_weights)
# Category probability dataframe
def calculate_category_probabilities(data_frame, category_column):
    # Count the number of rows for each category
    category_counts = data_frame[category_column].value_counts()

    # Calculate probabilities for each category
    category_probabilities = category_counts / len(data_frame)

    # Create a DataFrame with category_probabilities
    category_probabilities_df = pd.DataFrame({
        category_column: category_probabilities.index,
        'Probability': category_probabilities.values
    })

    return category_counts, category_probabilities_df

# Example usage:
# Assuming 'Data_binary' is your DataFrame with 'Category' column
category_counts, category_probabilities_df = calculate_category_probabilities(Data_binary, 'Category')
#Record probability per category
categories = np.sort(Data_binary['Category'].unique())

# Set constant values
record_prob = 1 / len(Data_binary)

# Calculate record_prob / P(c) for each category
record_prob_per_category = pd.DataFrame({
    'Category': categories,
    'Record_Prob_Per_Category': record_prob / category_probabilities_df.loc[category_probabilities_df['Category'].isin(categories), 'Probability'].values
})
# Maximum Posterior category
def calculate_max_posterior_category(binary_dataset, x_by_c_values, record_prob_per_category):
    predicted_categories = []
    posterior_probabilities = []

    for index, row_values in x_by_c_values.iterrows():
        # Extract the 'P(x/c)' value
        p_x_c = row_values['P(x/c)']

        # Calculate P(c/x) for each category
        category_probs = p_x_c * record_prob_per_category['Record_Prob_Per_Category']

        # Determine the predicted category as the one with the maximum P(c/x)
        predicted_category = record_prob_per_category['Category'][category_probs.idxmax()]
        predicted_categories.append(predicted_category)

        # Store the maximum posterior probability for the row
        max_posterior_probability = category_probs.max()
        posterior_probabilities.append(max_posterior_probability)

    # Add columns to the binary_dataset DataFrame
    binary_dataset['Predicted_Category'] = predicted_categories
    #binary_dataset['Posterior_Probability'] = posterior_probabilities

    return binary_dataset

# Example usage:
ankit_result = calculate_max_posterior_category(Data_binary, x_by_c_values, record_prob_per_category)
# Train-Test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(Data_binary, Data_binary[dependent], test_size=0.3, random_state=42)
#Calculate Root Mean Square Error
def calculate_rmse(X_test, intervals_df):
    # Calculate predicted category
    prediction = calculate_max_posterior_category(Data_binary, x_by_c_values, record_prob_per_category)
    predicted_column = prediction['Predicted_Category']

    # Initialize squared differences array
    squared_differences = np.zeros(len(X_test))
    sum = 0.0
    # Iterate over rows of X_test
    for i, row in X_test.iterrows():
        # Extract actual and predicted categories for the current row
        actual_category = row['Category']
        predicted_category = predicted_column[i]

        # Extract the max values for the actual and predicted categories
        actual_max_value = intervals_df[intervals_df['Category'] == actual_category]['Max_Value'].values[0]
        predicted_max_value = intervals_df[intervals_df['Category'] == predicted_category]['Max_Value'].values[0]

        # Calculate squared difference for the current row
        dif = predicted_max_value - actual_max_value
        sum+=dif;

    # Calculate mean squared error
    mean_squared_error = sum/len(X_test)

    # Calculate root mean squared error
    rmse = np.sqrt(mean_squared_error)

    return rmse

# Assuming X_test is your DataFrame
result_rmse = calculate_rmse(X_test, intervals_df)
print("Root Mean Squared Error (RMSE):", result_rmse)

Enter the path to the dataset CSV file: /content/drive/MyDrive/bolts.csv
Enter the name of the dependent column: T20BOLT
Enter the path to the frequent itemset CSV file: /content/drive/MyDrive/freq_item_bolts.csv
Root Mean Squared Error (RMSE): 0.7478076799715105


# Checking Aneesh Sir's conditions

In [31]:
def create_learning_curves(X_train, y_train, X_val, y_val, X_test, y_test, intervals_df, num_intervals):
    # Initialize lists to store training and validation errors
    train_errors = []
    val_errors = []

    # Train the model on varying sizes of the training set
    for i in range(10, len(X_train) + 1, 10):
        # Train the model on the first i samples
        X_train_subset = X_train.iloc[:i]
        y_train_subset = y_train.iloc[:i]

        # Train the model on the subset
        x_by_c_values_subset = calculate_x_by_c(X_train_subset, one_hot_encoded, categories_weights)

        # Calculate RMSE on the validation set
        val_rmse = calculate_rmse(X_val, intervals_df)

        # Calculate RMSE on the training set
        train_rmse = calculate_rmse(X_train_subset, intervals_df)

        # Append errors to the lists
        train_errors.append(train_rmse)
        val_errors.append(val_rmse)

    # Plot the learning curves
    plt.plot(range(10, len(X_train) + 1, 10), train_errors, label='Training RMSE')
    plt.plot(range(10, len(X_train) + 1, 10), val_errors, label='Validation RMSE')
    plt.xlabel('Number of Training Samples')
    plt.ylabel('RMSE')
    plt.title('Learning Curves')
    plt.legend()
    plt.show()

# Call the function to create learning curves
create_learning_curves(X_train, y_train, X_val, y_val, X_test, y_test, intervals_df, num_intervals)


KeyError: 15